In [878]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn


In [879]:
import torch
import torch.nn as nn
import torch.optim as optim

class ModelSharpe(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ModelSharpe, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def _sharpe_loss(self, portfolio_returns):
        mean_returns = portfolio_returns.mean(dim=1)
        std_returns = portfolio_returns.std(dim=1) + 1e-6
        sharpe_ratio = mean_returns / std_returns
        return -sharpe_ratio.mean()

    def forward(self, x, y):
        output_lstm, (hn, cn) = self.lstm(x)
        unnormalized_weights = self.linear(output_lstm)
        weights = torch.softmax(unnormalized_weights, dim=-1)
        portfolio_returns = torch.sum(weights[:, :, :] * y[:, :, :], dim=2)
        loss = self._sharpe_loss(portfolio_returns)
        return loss

    def get_allocations(self, x):
        with torch.no_grad():
            output_lstm, (hn, cn) = self.lstm(x)
            unnormalized_weights = self.linear(output_lstm)
            weights = torch.softmax(unnormalized_weights, dim=-1)
        return weights[:, -1, :]


In [880]:
random_tensor = 2 * torch.rand(1, 50, 5) - 1
y = 2 * torch.rand(1, 50, 5) - 1

In [881]:
model_lstm = ModelSharpe(5, 64, 5)

In [882]:
A = model_lstm(random_tensor, y)
print(A.shape, A)
print(model_lstm.get_allocations(random_tensor))

torch.Size([]) tensor(0.1566, grad_fn=<NegBackward0>)
tensor([[0.1946, 0.2059, 0.2095, 0.2062, 0.1839]])


In [883]:
import numpy as np
import yfinance as yf
import pandas as pd

tickers = ['VTI', 'AGG', 'DBC', '^VIX']
data = yf.download(tickers, start="2006-01-01", end="2020-12-31", interval="1d")['Adj Close']
data_na = data.dropna(axis = 0)
for column in data_na.columns:
    data_na[f'{column}_R'] = data_na[f'{column}'].pct_change()
    data_na[f'{column}_y'] = data_na[f'{column}_R'].shift(-1)
data_na.dropna(axis=0, inplace=True)
data_na.reset_index(inplace=True)
data_na['Date'] = data_na['Date'].dt.date
data_na

[*********************100%***********************]  4 of 4 completed
/var/folders/7q/v0vj3h051dn717mx7ngrr8yh0000gn/T/ipykernel_75850/2234399925.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_na[f'{column}_R'] = data_na[f'{column}'].pct_change()
/var/folders/7q/v0vj3h051dn717mx7ngrr8yh0000gn/T/ipykernel_75850/2234399925.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_na[f'{column}_y'] = data_na[f'{column}_R'].shift(-1)
/var/folders/7q/v0vj3h051dn717mx7ngrr8yh0000gn/T/ipykernel_75850/2234

Ticker,Date,AGG,DBC,VTI,^VIX,AGG_R,AGG_y,DBC_R,DBC_y,VTI_R,VTI_y,^VIX_R,^VIX_y
0,2006-02-07,56.438553,20.285255,44.219513,13.590000,-0.000699,-0.000499,-0.028926,-0.004255,-0.009736,0.007194,0.042178,-0.055923
1,2006-02-08,56.410397,20.198935,44.537624,12.830000,-0.000499,0.000599,-0.004255,0.009402,0.007194,-0.001905,-0.055923,0.022603
2,2006-02-09,56.444168,20.388840,44.452793,13.120000,0.000599,-0.002096,0.009402,-0.018205,-0.001905,0.002067,0.022603,-0.019055
3,2006-02-10,56.325878,20.017660,44.544689,12.870000,-0.002096,0.000700,-0.018205,-0.015524,0.002067,-0.004523,-0.019055,0.037296
4,2006-02-13,56.365326,19.706909,44.343224,13.350000,0.000700,-0.001600,-0.015524,-0.008323,-0.004523,0.009485,0.037296,-0.082397
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,2020-12-22,106.759201,13.624198,181.857040,24.230000,0.001443,-0.000678,-0.011004,0.012517,0.000207,0.001710,-0.036963,-0.037969
3746,2020-12-23,106.686790,13.794738,182.168060,23.309999,-0.000678,0.000933,0.012517,0.002747,0.001710,0.001673,-0.037969,-0.076362
3747,2020-12-24,106.786339,13.832635,182.472778,21.530001,0.000933,0.000170,0.002747,-0.006164,0.001673,0.006327,-0.076362,0.007896
3748,2020-12-28,106.804451,13.747366,183.627289,21.700001,0.000170,0.000170,-0.006164,0.002757,0.006327,-0.004174,0.007896,0.063594


In [884]:
def create_batches(dataframe, start_date, end_date, window_size=50):
    dataframe['Date'] = pd.to_datetime(dataframe['Date'])
    
    filtered_data = dataframe[(dataframe['Date'] >= start_date) & (dataframe['Date'] <= end_date)]
    
    x_batches = []
    y_batches = []
    dates_batches = []
    dates_per_feature = []
    y = []

    for i in range(len(filtered_data) - window_size):
        window_returns = filtered_data.iloc[i:i+window_size][[col for col in dataframe.columns if col.endswith('_R')]].values
        window_prices = filtered_data.iloc[i:i+window_size][[col for col in dataframe.columns if col in ['AGG', 'DBC', 'VTI', '^VIX']]].values
        
        window_x = np.concatenate([window_returns, window_prices], axis=1)
        window_y = filtered_data.iloc[i:i+window_size][[col for col in dataframe.columns if col.endswith('_y')]].values
        
        window_dates_batches = filtered_data.iloc[i:i+window_size]['Date'].values
        dates_per_feature.append(filtered_data.iloc[i+window_size]['Date'].date()) 

        y.append(filtered_data.iloc[i+window_size][[col for col in dataframe.columns if col.endswith('_y')]].values)

        x_batches.append(window_x)
        y_batches.append(window_y)
        dates_batches.append(window_dates_batches)

    return np.array(x_batches), np.array(y_batches), np.array(y, dtype=np.float32), np.array(dates_batches), dates_per_feature

x_batches, y_batches, y, dates_batches, dates_per_feature = create_batches(data_na, start_date="2006-01-01", end_date="2020-12-29")
y = y.reshape(y.shape[0], 1, y.shape[1])

/var/folders/7q/v0vj3h051dn717mx7ngrr8yh0000gn/T/ipykernel_75850/2367803767.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Date'] = pd.to_datetime(dataframe['Date'])


In [885]:
data_na.head(51)

Ticker,Date,AGG,DBC,VTI,^VIX,AGG_R,AGG_y,DBC_R,DBC_y,VTI_R,VTI_y,^VIX_R,^VIX_y
0,2006-02-07,56.438553,20.285255,44.219513,13.59,-0.000699,-0.000499,-0.028926,-0.004255,-0.009736,0.007194,0.042178,-0.055923
1,2006-02-08,56.410397,20.198935,44.537624,12.83,-0.000499,0.000599,-0.004255,0.009402,0.007194,-0.001905,-0.055923,0.022603
2,2006-02-09,56.444168,20.388840,44.452793,13.12,0.000599,-0.002096,0.009402,-0.018205,-0.001905,0.002067,0.022603,-0.019055
3,2006-02-10,56.325878,20.017660,44.544689,12.87,-0.002096,0.000700,-0.018205,-0.015524,0.002067,-0.004523,-0.019055,0.037296
4,2006-02-13,56.365326,19.706909,44.343224,13.35,0.000700,-0.001600,-0.015524,-0.008323,-0.004523,0.009485,0.037296,-0.082397
5,2006-02-14,56.275124,19.542898,44.763840,12.25,-0.001600,0.001101,-0.008323,-0.014576,0.009485,0.003791,-0.082397,0.004898
6,2006-02-15,56.337093,19.258043,44.933517,12.31,0.001101,-0.000999,-0.014576,0.012102,0.003791,0.007788,0.004898,-0.067425
7,2006-02-16,56.280807,19.491108,45.283455,11.48,-0.000999,0.004105,0.012102,0.014172,0.007788,-0.001171,-0.067425,0.046167
8,2006-02-17,56.511845,19.767332,45.230442,12.01,0.004105,-0.003690,0.014172,0.016594,-0.001171,-0.002814,0.046167,0.033306
9,2006-02-21,56.303314,20.095350,45.103176,12.41,-0.003690,0.004004,0.016594,-0.016323,-0.002814,0.007837,0.033306,-0.042707


In [886]:
import datetime


def get_idx_dates(start_date, end_date):
    if start_date in dates_per_feature and end_date in dates_per_feature:
        k_start = dates_per_feature.index(start_date)
        k_end = dates_per_feature.index(end_date)
        return k_start, k_end
    
    if start_date not in dates_per_feature: 
        if any(dates >= start_date for dates in dates_per_feature):    
            k_start = min(j for j in range(len(dates_per_feature)) if dates_per_feature[j] >= start_date)
        else:
            k_start = 0
    else:
        k_start = dates_per_feature.index(start_date)

    if end_date not in dates_per_feature:
        if any(dates <= end_date for dates in dates_per_feature):
            k_end = max(j for j in range(len(dates_per_feature)) if dates_per_feature[j] <= end_date) + 1
        else:
            k_end = len(dates_per_feature) -1
    else: 
        k_end = dates_per_feature.index(end_date)

    return k_start, k_end




def training(x_batches, y_batches, model_lstm, batch_size):

    x_batches_tensor = torch.tensor(x_batches, dtype=torch.float32)
    y_batches_tensor = torch.tensor(y_batches, dtype=torch.float32)
    num_batches = x_batches.shape[0]
    optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
    num_epochs = 100


    for epoch in range(num_epochs):
        model_lstm.train()
        loss_epoch = 0
        
        for i in range(0, num_batches, batch_size):
            x_mini_batch = x_batches_tensor[i:i+batch_size]
            y_mini_batch = y_batches_tensor[i:i+batch_size]
            
            optimizer.zero_grad()
            
            loss = model_lstm(x_mini_batch, y_mini_batch)
            loss_epoch += loss.item()
            loss.backward()
            optimizer.step()
        
        print(f"epoch [{epoch+1}/{num_epochs}], loss: {(loss_epoch/batch_size)}")

    return model_lstm



def investing(x_batches, y, model_lstm):

    x_tensor = torch.tensor(x_batches, dtype=torch.float32)
    allocations = model_lstm.get_allocations(x_tensor)
    allocations = allocations.view(allocations.shape[0], 1, allocations.shape[1])
    rdt = torch.sum(allocations*y, dim=2)


    return rdt, allocations


In [887]:
y_real.shape

(179, 1, 4)

In [888]:
input_size = 8
hidden_size = 64
output_size = 4
batch_size = 64

rdt_all = torch.empty(0,0)
alloc_all = []
idx_invest = []
dates_invest = []


first_date = dates_per_feature[0]
end_date_1st_training = first_date + datetime.timedelta(days=365*4)
idx_start, idx_end = get_idx_dates(first_date, end_date_1st_training)
model_lstm = ModelSharpe(input_size=input_size, hidden_size=hidden_size, output_size=output_size)


model_lstm = training(x_batches = x_batches[idx_start:idx_end, :, :],
                      y_batches= y_batches[idx_start:idx_end, :, :],
                      model_lstm=model_lstm,
                      batch_size=64)

date_end_invest = end_date_1st_training + datetime.timedelta(days=365*2)
idx_start_invest, idx_end_invest = get_idx_dates(end_date_1st_training, date_end_invest)

print(f"Idx start: {idx_start_invest}, Idx end: {idx_end_invest}")
print(dates_per_feature[idx_start_invest], dates_per_feature[idx_end_invest])

idx_invest.append((idx_start_invest, idx_end_invest))
dates_invest.append((end_date_1st_training, date_end_invest))

x_investing = x_batches[idx_start_invest:idx_end_invest, :, :]
y_investing = y_batches[idx_start_invest:idx_end_invest, :, :]
y_real = y[idx_start_invest:idx_end_invest, :, :]


first_rdt, first_alloc = investing(x_batches= x_investing, y=y_real, model_lstm=model_lstm)

print(first_rdt.shape, len(dates_per_feature[idx_start_invest:idx_end_invest]))

print(f"first_rdt shape: {first_rdt.shape}, rdt_all shape before concat: {rdt_all.shape}")


rdt_all = first_rdt
alloc_all = first_alloc



for i in range(5):

    model_lstm = training(x_batches = x_batches[idx_start_invest:idx_end_invest, :, :],
                      y_batches= y_batches[idx_start_invest:idx_end_invest, :, :],
                      model_lstm=model_lstm,
                      batch_size=64)
    
    date_end_invest = date_end_invest + datetime.timedelta(days=365*2)
    end_date_1st_training = end_date_1st_training + datetime.timedelta(days=365*2)
    idx_start_invest, idx_end_invest = get_idx_dates(end_date_1st_training, date_end_invest)

    x_investing = x_batches[idx_start_invest:idx_end_invest, :, :]
    y_investing = y_batches[idx_start_invest:idx_end_invest, :, :]
    y_real = y[idx_start_invest:idx_end_invest, :, :]


    first_rdt, first_alloc = investing(x_batches= x_investing, y=y_real, model_lstm=model_lstm)

    print(first_rdt.shape, len(dates_per_feature[idx_start_invest:idx_end_invest]))


    rdt_all = torch.cat([rdt_all, first_rdt], dim=0)
    print('len des rdt', rdt_all.shape[0])
    alloc_all = torch.cat([alloc_all, first_alloc], dim=0)
    idx_invest.append((idx_start_invest, idx_end_invest))
    print('len des dates', len(dates_per_feature[idx_invest[0][0]:idx_invest[-1][1]]))
    dates_invest.append((end_date_1st_training, date_end_invest))


alloc_all = alloc_all.numpy()
alloc_all = alloc_all.reshape(alloc_all.shape[0], alloc_all.shape[2])






epoch [1/100], loss: -0.016230566310696304
epoch [2/100], loss: -0.024055742338532582
epoch [3/100], loss: -0.030460993613814935
epoch [4/100], loss: -0.03354835053323768
epoch [5/100], loss: -0.03665075410390273
epoch [6/100], loss: -0.04048267315374687
epoch [7/100], loss: -0.041557419681339525
epoch [8/100], loss: -0.0443720019684406
epoch [9/100], loss: -0.04531284759286791
epoch [10/100], loss: -0.04781841034855461
epoch [11/100], loss: -0.047203864241964766
epoch [12/100], loss: -0.04854617234377656
epoch [13/100], loss: -0.048483106886124006
epoch [14/100], loss: -0.050267823033209424
epoch [15/100], loss: -0.051385304584982805
epoch [16/100], loss: -0.05097095114979311
epoch [17/100], loss: -0.05114922947905143
epoch [18/100], loss: -0.052019179638591595
epoch [19/100], loss: -0.05244371379376389
epoch [20/100], loss: -0.050471387250581756
epoch [21/100], loss: -0.05236392089864239
epoch [22/100], loss: -0.054288732848363
epoch [23/100], loss: -0.054482774809002876
epoch [24/10

/var/folders/7q/v0vj3h051dn717mx7ngrr8yh0000gn/T/ipykernel_75850/2316995471.py:66: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  rdt = torch.sum(allocations*y, dim=2)


epoch [2/100], loss: -0.03423302015289664
epoch [3/100], loss: -0.036298931343480945
epoch [4/100], loss: -0.03631727397441864
epoch [5/100], loss: -0.037152034463360906
epoch [6/100], loss: -0.03789276350289583
epoch [7/100], loss: -0.03759399219416082
epoch [8/100], loss: -0.03909530327655375
epoch [9/100], loss: -0.03866000799462199
epoch [10/100], loss: -0.039096108404919505
epoch [11/100], loss: -0.03596768924035132
epoch [12/100], loss: -0.03838262194767594
epoch [13/100], loss: -0.03979859501123428
epoch [14/100], loss: -0.04041334264911711
epoch [15/100], loss: -0.04069394967518747
epoch [16/100], loss: -0.0406706603243947
epoch [17/100], loss: -0.04217653814703226
epoch [18/100], loss: -0.0412274319678545
epoch [19/100], loss: -0.04135071043856442
epoch [20/100], loss: -0.04306741803884506
epoch [21/100], loss: -0.04330442403443158
epoch [22/100], loss: -0.04267962300218642
epoch [23/100], loss: -0.04283900395967066
epoch [24/100], loss: -0.04309803317300975
epoch [25/100], lo

In [928]:

dataframe_result = pd.DataFrame({
    'Date': pd.Series(dates_per_feature[idx_invest[0][0]:idx_invest[-1][1]]),
    'Return': pd.Series(rdt_all.numpy().flatten()),
    'Alloc_AGG': pd.Series(alloc_all[:, 0]),
    'Alloc_DBC': pd.Series(alloc_all[:, 1]),
    'Alloc_VTI': pd.Series(alloc_all[:, 2]),
    'Alloc_^VIX': pd.Series(alloc_all[:, 3])
})

dataframe_result['Date'] = pd.to_datetime(dataframe_result['Date'])
data_na['Date'] = pd.to_datetime(data_na['Date'])
dataframe_result = pd.merge(dataframe_result, data_na[['Date', 'AGG_y', 'DBC_y', 'VTI_y', '^VIX_y']], on='Date', how='left')

dataframe_result['Final_Return'] = (
    dataframe_result['Alloc_AGG'] * dataframe_result['AGG_y'] +
    dataframe_result['Alloc_DBC'] * dataframe_result['DBC_y'] +
    dataframe_result['Alloc_VTI'] * dataframe_result['VTI_y'] +
    dataframe_result['Alloc_^VIX'] * dataframe_result['^VIX_y']
)

dataframe_result

/var/folders/7q/v0vj3h051dn717mx7ngrr8yh0000gn/T/ipykernel_75850/2977585624.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_na['Date'] = pd.to_datetime(data_na['Date'])


,Date,Return,Alloc_AGG,Alloc_DBC,Alloc_VTI,Alloc_^VIX,AGG_y,DBC_y,VTI_y,^VIX_y,Final_Return
0,2010-04-19,0.002481,0.526954,0.005940,0.439238,0.027868,0.001631,0.007937,0.009474,-0.092849,0.002481
1,2010-04-20,0.002020,0.603642,0.007728,0.355869,0.032760,0.001724,0.004973,-0.000810,0.037508,0.002020
2,2010-04-21,0.001406,0.532453,0.008967,0.406784,0.051796,-0.002008,0.002474,0.004859,0.009191,0.001406
3,2010-04-22,0.003090,0.472861,0.007492,0.471515,0.048132,-0.001246,0.006582,0.006768,0.009108,0.003090
4,2010-04-23,0.000492,0.478405,0.007210,0.470758,0.043627,-0.000096,-0.004904,-0.003521,0.051143,0.000492
...,...,...,...,...,...,...,...,...,...,...,...
2690,2020-12-22,-0.000674,0.996573,0.000181,0.003087,0.000159,-0.000678,0.012517,0.001710,-0.037969,-0.000674
2691,2020-12-23,0.000921,0.996361,0.000219,0.003224,0.000196,0.000933,0.002747,0.001673,-0.076362,0.000921
2692,2020-12-24,0.000196,0.995157,0.000330,0.004231,0.000282,0.000170,-0.006164,0.006327,0.007896,0.000196
2693,2020-12-28,0.000178,0.990910,0.000769,0.007688,0.000633,0.000170,0.002757,-0.004174,0.063594,0.000178


In [930]:
dataframe_result.describe()

,Date,Return,Alloc_AGG,Alloc_DBC,Alloc_VTI,Alloc_^VIX,AGG_y,DBC_y,VTI_y,^VIX_y,Final_Return
count,2695,2695.000000,2695.000000,2.695000e+03,2695.000000,2695.000000,2695.000000,2695.000000,2695.000000,2695.000000,2695.000000
mean,2015-08-23 09:22:06.456400640,0.001232,0.486613,4.459733e-02,0.349904,0.118886,0.000156,-0.000119,0.000567,0.003328,0.001232
min,2010-04-19 00:00:00,-0.128549,0.050967,1.209566e-07,0.002127,0.000015,-0.040010,-0.077844,-0.113808,-0.295727,-0.128549
25%,2012-12-18 12:00:00,-0.001669,0.208276,2.861310e-04,0.216522,0.012359,-0.001085,-0.005482,-0.003573,-0.043433,-0.001669
50%,2015-08-24 00:00:00,0.000401,0.509281,5.901064e-03,0.307070,0.044556,0.000254,0.000379,0.000771,-0.005657,0.000401
75%,2018-04-26 12:00:00,0.002486,0.707553,1.952869e-02,0.479950,0.087753,0.001436,0.005494,0.005588,0.035866,0.002486
max,2020-12-29 00:00:00,0.625255,0.997802,5.673330e-01,0.818960,0.724120,0.023721,0.044342,0.094898,1.155979,0.625255
std,NaN,0.020281,0.289234,1.120117e-01,0.205815,0.193734,0.002521,0.010216,0.011177,0.083941,0.020281


In [890]:
trading_days_per_year = 252

mean_daily_return = dataframe_result['Final_Return'].mean()
annualized_return = (1+mean_daily_return) ** trading_days_per_year -1

std_daily_return = dataframe_result['Final_Return'].std()
std_dd_return = dataframe_result[dataframe_result['Final_Return'] < 0]['Final_Return'].std()

annualized_volatility = std_daily_return * np.sqrt(trading_days_per_year)
annualized_volatility_dd = std_dd_return * np.sqrt(trading_days_per_year)

sharpe_ratio = annualized_return / annualized_volatility
sortino_ratio = annualized_return / annualized_volatility_dd

print(f"annualized mean return: {annualized_return:.4f}")
print(f"annualized std: {annualized_volatility:.4f}")
print(f"annualized dd: {annualized_volatility_dd:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Sortino Ratio: {sortino_ratio:.4f}")


annualized mean return: 0.3639
annualized std: 0.3220
annualized dd: 0.1776
Sharpe Ratio: 1.1304
Sortino Ratio: 2.0497


In [936]:
assets = ['AGG', 'DBC', 'VTI', '^VIX']  # Liste des colonnes actifs de 'data_na'
for asset in assets:
    dataframe_result[f'{asset}_ewm_std_50d'] = None  # Volatilité exponentiellement pondérée sur les 50 derniers jours
    dataframe_result[f'{asset}_ewm_std_skip_50d'] = None  # Volatilité sur 50 jours avant en sautant jour courant et jour-1
    dataframe_result[f'{asset}_std_50d'] = None  # Volatilité classique (écart-type) sur les 50 derniers jours
    dataframe_result[f'{asset}_std_skip_50d'] = None  # Volatilité classique sur 50 jours en sautant le jour courant et jour-1

# Calcul de la volatilité exponentiellement pondérée et classique
for i, row in dataframe_result.iterrows():
    current_date = row['Date']
    
    # Filtrer les 50 derniers jours avant la date actuelle
    mask = (data_na['Date'] < current_date)
    previous_50_days = data_na.loc[mask].tail(50)  # Derniers 50 jours sans inclure jour actuel
    
    # Calcul de la volatilité exponentiellement pondérée et classique sur les 50 derniers jours
    for asset in assets:
        # Volatilité exponentiellement pondérée
        ewm_std_50d = previous_50_days[f"{asset}_R"].ewm(span=50, adjust=False).std().iloc[-1]
        dataframe_result.at[i, f'{asset}_ewm_std_50d'] = ewm_std_50d
        
        # Volatilité classique (écart-type simple)
        std_50d = previous_50_days[f"{asset}_R"].std()
        dataframe_result.at[i, f'{asset}_std_50d'] = std_50d
    
    # Filtrer pour les 50 jours avant en sautant le jour actuel et jour-1
    mask_skip = (data_na['Date'] < current_date)
    days_to_skip = data_na.loc[mask_skip].tail(51).head(50)  # Sauter le jour actuel et jour-1
    for asset in assets:
        # Volatilité exponentiellement pondérée en sautant jour actuel et jour-1
        ewm_std_skip_50d = days_to_skip[f"{asset}_R"].ewm(span=50, adjust=False).std().iloc[-1]
        dataframe_result.at[i, f'{asset}_ewm_std_skip_50d'] = ewm_std_skip_50d
        
        # Volatilité classique (écart-type simple) en sautant jour actuel et jour-1
        std_skip_50d = days_to_skip[f"{asset}_R"].std()
        dataframe_result.at[i, f'{asset}_std_skip_50d'] = std_skip_50d


In [941]:
dataframe_result.mean()

Date                     2015-08-23 09:22:06.456400640
Return                                        0.001232
Alloc_AGG                                     0.486613
Alloc_DBC                                     0.044597
Alloc_VTI                                     0.349904
Alloc_^VIX                                    0.118886
AGG_y                                         0.000156
DBC_y                                        -0.000119
VTI_y                                         0.000567
^VIX_y                                        0.003328
Final_Return                                  0.001232
AGG_ewm_std_50d                               0.002137
AGG_ewm_std_skip_50d                          0.002137
AGG_std_50d                                   0.002148
AGG_std_skip_50d                              0.002148
DBC_ewm_std_50d                               0.009666
DBC_ewm_std_skip_50d                          0.009667
DBC_std_50d                                   0.009718
DBC_std_sk

In [892]:
dataframe_result_scaled = dataframe_result.iloc[50:,:]

In [949]:
C = 0.0001
vol_t = 0.000378

# Calcul du retour pondéré par la volatilité de chaque actif
agg_return = (vol_t / dataframe_result_scaled['AGG_ewm_std_50d']) * (dataframe_result_scaled['Alloc_AGG'] * dataframe_result_scaled['AGG_y'])
dbc_return = (vol_t / dataframe_result_scaled['DBC_ewm_std_50d']) * (dataframe_result_scaled['Alloc_DBC'] * dataframe_result_scaled['DBC_y'])
vti_return = (vol_t / dataframe_result_scaled['VTI_ewm_std_50d']) * (dataframe_result_scaled['Alloc_VTI'] * dataframe_result_scaled['VTI_y'])
vix_return = (vol_t / dataframe_result_scaled['^VIX_ewm_std_50d']) * (dataframe_result_scaled['Alloc_^VIX'] * dataframe_result_scaled['^VIX_y'])

# Calcul de la somme des retours pondérés
total_return = agg_return + dbc_return + vti_return + vix_return

# Calcul de la pénalité pour les variations d'allocation
agg_penalty = np.abs((vol_t / dataframe_result_scaled['AGG_ewm_std_50d']) * dataframe_result_scaled['Alloc_AGG'] - 
                     (vol_t / dataframe_result_scaled['AGG_ewm_std_50d'].shift(1)) * dataframe_result_scaled['Alloc_AGG'].shift(1))

dbc_penalty = np.abs((vol_t / dataframe_result_scaled['DBC_ewm_std_50d']) * dataframe_result_scaled['Alloc_DBC'] - 
                     (vol_t / dataframe_result_scaled['DBC_ewm_std_50d'].shift(1)) * dataframe_result_scaled['Alloc_DBC'].shift(1))

vti_penalty = np.abs((vol_t / dataframe_result_scaled['VTI_ewm_std_50d']) * dataframe_result_scaled['Alloc_VTI'] - 
                     (vol_t / dataframe_result_scaled['VTI_ewm_std_50d'].shift(1)) * dataframe_result_scaled['Alloc_VTI'].shift(1))

vix_penalty = np.abs((vol_t / dataframe_result_scaled['^VIX_ewm_std_50d']) * dataframe_result_scaled['Alloc_^VIX'] - 
                     (vol_t / dataframe_result_scaled['^VIX_ewm_std_50d'].shift(1)) * dataframe_result_scaled['Alloc_^VIX'].shift(1))

# Somme des pénalités
total_penalty = agg_penalty + dbc_penalty + vti_penalty + vix_penalty

# Calcul final avec la pénalité
dataframe_result_scaled['Final_Return_scaled_C'] = total_return - C * total_penalty


/var/folders/7q/v0vj3h051dn717mx7ngrr8yh0000gn/T/ipykernel_75850/1352436497.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_result_scaled['Final_Return_scaled_C'] = total_return - C * total_penalty


In [943]:
dataframe_result_scaled.head(50)

,Date,Return,Alloc_AGG,Alloc_DBC,Alloc_VTI,Alloc_^VIX,AGG_y,DBC_y,VTI_y,^VIX_y,Final_Return,AGG_ewm_std_50d,AGG_ewm_std_skip_50d,DBC_ewm_std_50d,DBC_ewm_std_skip_50d,VTI_ewm_std_50d,VTI_ewm_std_skip_50d,^VIX_ewm_std_50d,^VIX_ewm_std_skip_50d,Final_Return_scaled_C
50,2010-06-29,-0.002003,0.786345,0.005715,0.200423,0.007516,0.000093,0.003723,-0.010914,0.012013,-0.002003,0.001944,0.00178,0.012907,0.013825,0.015343,0.016356,0.103248,0.115278,NaN
51,2010-06-30,-0.001823,0.712532,0.003938,0.280175,0.003354,-0.000524,-0.015299,-0.004376,-0.048639,-0.001823,0.001771,0.001944,0.013509,0.012907,0.016665,0.015343,0.110883,0.103248,-0.00011
52,2010-07-01,-0.002482,0.747056,0.004294,0.245629,0.003021,-0.001404,-0.001883,-0.004778,-0.083384,-0.002482,0.00175,0.001771,0.013044,0.013509,0.01584,0.016665,0.101928,0.110883,-0.000257
53,2010-07-02,0.003019,0.766269,0.005025,0.225110,0.003596,0.003092,0.002830,0.003073,-0.015604,0.003019,0.001949,0.00175,0.01296,0.013044,0.015773,0.01584,0.100516,0.101928,0.000475
54,2010-07-06,0.005258,0.783644,0.006009,0.204860,0.005487,-0.001307,0.022578,0.032542,-0.094772,0.005258,0.00183,0.001949,0.012974,0.01296,0.015609,0.015773,0.100377,0.100516,-0.000049
55,2010-07-07,0.001706,0.775242,0.006052,0.211908,0.006799,-0.000187,0.009200,0.009826,-0.042101,0.001706,0.001763,0.00183,0.012401,0.012974,0.014921,0.015609,0.099627,0.100377,0.000022
56,2010-07-08,-0.000176,0.800917,0.007779,0.178387,0.012918,-0.001497,0.001823,0.007711,-0.028394,-0.000176,0.001955,0.001763,0.013967,0.012401,0.018027,0.014921,0.144349,0.099627,-0.000205
57,2010-07-09,0.000460,0.795471,0.008097,0.177928,0.018504,0.001125,-0.011374,0.000364,-0.022018,0.000460,0.00187,0.001955,0.012995,0.013967,0.016228,0.018027,0.098039,0.144349,0.000178
58,2010-07-12,0.001163,0.791653,0.008192,0.177655,0.022500,-0.002527,0.016107,0.016390,0.005321,0.001163,0.001725,0.00187,0.013004,0.012995,0.016497,0.016228,0.102197,0.098039,-0.000369
59,2010-07-13,0.002952,0.785875,0.008251,0.180174,0.025700,0.003377,-0.001812,-0.000179,0.013436,0.002952,0.00214,0.001725,0.012963,0.013004,0.016449,0.016497,0.113537,0.102197,0.000465


In [946]:
mean_daily_return_scaled = dataframe_result_scaled['Final_Return_scaled_C'].mean()
annualized_return_scaled = (1+mean_daily_return_scaled) ** trading_days_per_year -1

std_daily_return_scaled = dataframe_result_scaled['Final_Return_scaled_C'].std()
std_dd_return_scaled = dataframe_result_scaled[dataframe_result_scaled['Final_Return_scaled_C'] < 0]['Final_Return_scaled_C'].std()

annualized_volatility_scaled = std_daily_return * np.sqrt(trading_days_per_year)
annualized_volatility_dd_scaled = std_dd_return * np.sqrt(trading_days_per_year)

sharpe_ratio_scaled = annualized_return / annualized_volatility
sortino_ratio_scaled = annualized_return / annualized_volatility_dd

print(f"annualized mean return: {annualized_return_scaled:.4f}")
print(f"annualized std: {annualized_volatility_scaled:.4f}")
print(f"annualized dd: {annualized_volatility_dd_scaled:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio_scaled:.4f}")
print(f"Sortino Ratio: {sortino_ratio_scaled:.4f}")

annualized mean return: 0.0059
annualized std: 0.3220
annualized dd: 0.1776
Sharpe Ratio: 1.1304
Sortino Ratio: 2.0497


np.float64(2.329303170028373e-05)

In [872]:
mean_daily_return

np.float64(0.000882290624227052)

In [873]:
mean_daily_return_scaled

np.float64(0.006525504109762455)